In [ ]:
!pip install snorkel

In [7]:
!rm -rf data *.zip
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00380/YouTube-Spam-Collection-v1.zip
!tar -xf YouTube-Spam-Collection-v1.zip
!mkdir data
!mv Youtube*.csv data
!rm YouTube-Spam-Collection-v1.zip

--2020-09-27 20:11:03--  http://archive.ics.uci.edu/ml/machine-learning-databases/00380/YouTube-Spam-Collection-v1.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 163567 (160K) [application/x-httpd-php]
Saving to: 'YouTube-Spam-Collection-v1.zip'

     0K .......... .......... .......... .......... .......... 31%  258K 0s
    50K .......... .......... .......... .......... .......... 62%  497K 0s
   100K .......... .......... .......... .......... .......... 93% 13.2M 0s
   150K .........                                             100% 12.8M=0.3s

2020-09-27 20:11:04 (534 KB/s) - 'YouTube-Spam-Collection-v1.zip' saved [163567/163567]



In [21]:
import snorkel
from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis
from snorkel.analysis import get_label_buckets
from utils import load_spam_dataset

In [10]:
df_train, df_test = load_spam_dataset()

# We pull out the label vectors for ease of use later
Y_test = df_test.label.values

In [11]:
df_train.head()

,author,date,text,label,video
0,Alessandro leite,2014-11-05T22:21:36,pls http://www10.vakinha.com.br/VaquinhaE.aspx...,-1.0,1
1,Salim Tayara,2014-11-02T14:33:30,"if your like drones, plz subscribe to Kamal Ta...",-1.0,1
2,Phuc Ly,2014-01-20T15:27:47,go here to check the views :3﻿,-1.0,1
3,DropShotSk8r,2014-01-19T04:27:18,"Came here to check the views, goodbye.﻿",-1.0,1
4,css403,2014-11-07T14:25:48,"i am 2,126,492,636 viewer :D﻿",-1.0,1


In [12]:
ABSTAIN = -1
HAM = 0
SPAM = 1

In [16]:
@labeling_function()
def check(x):
    return SPAM if "check" in x.text.lower() else ABSTAIN

@labeling_function()
def check_out(x):
    return SPAM if "check out" in x.text.lower() else ABSTAIN

lfs = [ check_out, check ]
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

C:\Users\bill\software\anaconda3\lib\site-packages\tqdm\std.py:656: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|███████████████████████████████████████████████████████████████████████████| 1586/1586 [00:00<00:00, 31713.23it/s]


In [17]:
coverage_check_out, coverage_check = (L_train != ABSTAIN).mean(axis=0)
print(f"check_out coverage: {coverage_check_out * 100:.1f}%")
print(f"check coverage: {coverage_check * 100:.1f}%")

check_out coverage: 21.4%
check coverage: 25.8%


In [19]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check_out,0,[1],0.214376,0.214376,0.0
check,1,[1],0.257881,0.214376,0.0
